In [1]:
import spacy
from spacy import displacy
import pandas as pd
import re
import itertools as it
import os
import random
from spacy.training import offsets_to_biluo_tags,biluo_tags_to_spans
from spacy.tokens import Doc, DocBin
from ast import literal_eval

In [2]:
try:
    from collections.abc import Mapping
except ImportError:
    from collections import Mapping

In [59]:
file=pd.read_csv("RAW_recipes.csv")
file.head(5)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,16/09/2005,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,17/06/2002,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,25/02/2005,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,14/04/2003,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,25/10/2002,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [60]:
#preprocessing ingredients and raw recipies columns
#converting ingredients to a list of lists, then flattening the ingredients list, to create a single list containing all ingredients
file["ingredients"]= file["ingredients"].apply(lambda x: literal_eval(x))
ingredients= file["ingredients"].tolist()
ingredients

[['winter squash',
  'mexican seasoning',
  'mixed spice',
  'honey',
  'butter',
  'olive oil',
  'salt'],
 ['prepared pizza crust',
  'sausage patty',
  'eggs',
  'milk',
  'salt and pepper',
  'cheese'],
 ['ground beef',
  'yellow onions',
  'diced tomatoes',
  'tomato paste',
  'tomato soup',
  'rotel tomatoes',
  'kidney beans',
  'water',
  'chili powder',
  'ground cumin',
  'salt',
  'lettuce',
  'cheddar cheese'],
 ['spreadable cheese with garlic and herbs',
  'new potatoes',
  'shallots',
  'parsley',
  'tarragon',
  'olive oil',
  'red wine vinegar',
  'salt',
  'pepper',
  'red bell pepper',
  'yellow bell pepper'],
 ['tomato juice',
  'apple cider vinegar',
  'sugar',
  'salt',
  'pepper',
  'clove oil',
  'cinnamon oil',
  'dry mustard'],
 ['milk', 'vanilla ice cream', 'frozen apple juice concentrate', 'apple'],
 ['fennel seeds',
  'green olives',
  'ripe olives',
  'garlic',
  'peppercorn',
  'orange rind',
  'orange juice',
  'red chile',
  'extra virgin olive oil'],
 [

In [61]:
ingredients = list(set(it.chain(*ingredients)))#flattening the list, then converting it to a set to remove duplicates, and converting it to a list again
ingredients

['sherry wine vinegar',
 'four cheese blend',
 'anaheim chilies',
 'pak choi',
 'fresh nutmeg',
 'coffee-mate cinnamon vanilla liquid creamer',
 'madeira wine',
 'dry roasted salted peanuts',
 'palm oil',
 'ground pecans',
 'grapefruit zest',
 'flaked sea salt',
 'lemon verbena leaf',
 "eggland's best large egg",
 'white vinegar',
 'nacho chip',
 'raw peanuts',
 'strawberry vodka',
 'karo syrup',
 'cilantro leaves and stems',
 'beef bouillon paste',
 'rump roast',
 'lemon pudding mix',
 'raw sugar',
 'guar gum',
 'boneless eye of round beef steak',
 'low sodium chicken broth',
 'sugar-free applesauce',
 'boston lettuce',
 'blade steaks',
 'macadamia nuts',
 'dried apple',
 'rosemary sprigs',
 'taco sauce',
 '2% evaporated milk',
 'brown lentils',
 'orange syrup',
 'frozen waffles',
 'hot pepper oil',
 'dried red pepper flakes',
 'fat-free mayonnaise',
 'roasting chickens',
 'low-fat monterey jack pepper cheese',
 'tiger shrimp',
 'citrus juice',
 'sugar-free instant pudding mix',
 'sal

In [62]:
#doing the same with steps
file["steps"]= file["steps"].apply(lambda x: literal_eval(x))
steps = file["steps"].tolist()
steps = list(set(it.chain(*steps)))
steps

['bake cake until center is firm and cake tester comes out clean , 50 to 55 minutes',
 'sprinkle evenly with pepper',
 'sprouts are a great addition , sprinkled just before eating',
 'cut each tomato into 6 wedges and each artichoke into halves',
 'grill until desired "done-ness',
 'remove foil and bake for an additional 10 minutes',
 'press lightly onto bottom of pan using the back of a large metal spoon',
 "place 1 tablespoon hershey's butterscotch chips with 1 / 2 teaspoon shortening only , in a small microwave-safe bowl",
 'follow directions on the bakers dipping chocolate to melt the milk chocolate',
 'pour into glass and then add the grenadine until you see the sunrise',
 'add the potatoes and carrots and cook for about 5 minutes',
 'combine all other ingredients in a pot large enough to hold them , plus the apples',
 'store',
 'turn squash cut side up and cool',
 'bake for 30-35 minutes , or until cake tests done',
 'slice up bell peppers , add to mixture and saute an additional

In [63]:
#next we define a pattern, and train the nlp model with our defined pattern
pattern=re.compile(r'\b(?:%s)\b'% '|'.join(ingredients))
pattern

re.compile(r'\b(?:sherry wine vinegar|four cheese blend|anaheim chilies|pak choi|fresh nutmeg|coffee-mate cinnamon vanilla liquid creamer|madeira wine|dry roasted salted peanuts|palm oil|ground pecans|grapefruit zest|flaked sea salt|lemon verbena leaf|eggland\'s best large egg|white vinegar|nacho chip|raw peanuts|strawberry vodka|karo syrup|cilantro leaves and stems|beef bouillon paste|rump roast|lemon pudding mix|raw sugar|guar gum|boneless eye of round beef steak|low sodium chicken broth|sugar-free applesauce|boston lettuce|blade steaks|macadamia nuts|dried apple|rosemary sprigs|taco sauce|2% evaporated milk|brown lentils|orange syrup|frozen waffles|hot pepper oil|dried red pepper flakes|fat-free mayonnaise|roasting chickens|low-fat monterey jack pepper cheese|tiger shrimp|citrus juice|sugar-free instant pudding mix|salad supreme dry seasoning|toffee pieces|sesame dressing|red curry paste|dried mint|creme de noyaux|ground cardamom|melon liqueur|whole bay leaf|fresh chives|mixed candi

In [64]:
#next we create annotations, and search for pattern inside the text
annotations = []
annotation_text = []
for i in steps:
    match = pattern.finditer(i) #finditer gives all words match along with the span
    #next we create dictionaries
    temp1 = {}
    temp2 = {}
    val1 = []
    val2 = []
    for m in match:
        if m.group():
            val1.append([m.start(), m.end(), "Ingredients"]) #to get the start and end of each word span and append it to one list
            val2.append([m.group(),m.start(), m.end(), "Ingredients"])  # eher we append the llist with word and span, this is not required
        temp1.update({"Entities":val1}) #next we update our dictionary with the created files
        temp2.update({"Entities":val2})
        annotations.append([i, temp1])
        annotation_text.append([i,temp2])

In [65]:
annotation_text

[['bake cake until center is firm and cake tester comes out clean , 50 to 55 minutes',
  {'Entities': [['cake', 5, 9, 'Ingredients'],
    ['cake', 35, 39, 'Ingredients']]}],
 ['bake cake until center is firm and cake tester comes out clean , 50 to 55 minutes',
  {'Entities': [['cake', 5, 9, 'Ingredients'],
    ['cake', 35, 39, 'Ingredients']]}],
 ['sprinkle evenly with pepper',
  {'Entities': [['pepper', 21, 27, 'Ingredients']]}],
 ['cut each tomato into 6 wedges and each artichoke into halves',
  {'Entities': [['artichoke', 39, 48, 'Ingredients']]}],
 ["place 1 tablespoon hershey's butterscotch chips with 1 / 2 teaspoon shortening only , in a small microwave-safe bowl",
  {'Entities': [['butterscotch chips', 29, 47, 'Ingredients'],
    ['shortening', 68, 78, 'Ingredients']]}],
 ["place 1 tablespoon hershey's butterscotch chips with 1 / 2 teaspoon shortening only , in a small microwave-safe bowl",
  {'Entities': [['butterscotch chips', 29, 47, 'Ingredients'],
    ['shortening', 68, 78,

In [68]:
db = DocBin() #for serializing the model, to speed up the training process, by packing everything  into one object
#we split the dataset to train and val, tehn we seperate the text lines, and annotations
for text, annot in annotations[0:10000]: #keeping 1000 for testing
    try:
        docs = nlp.make_doc(text) #converting the text strings to normal one documnet
        tags = offsets_to_biluo_tags(docs, annot["Entities"]) # we add the bilou tags to words
        ents = biluo_tags_to_spans(docs, tags) # then find their spans
        docs.ents = ents
        db.add(docs)
    except IndexError:
        pass #if any statement is giving error pass will not consider that
    db.to_disk("train.spacy") 

In [70]:
for text, annot in annotations[10000:12000]: #creating for val
    try:
        doc = nlp.make_doc(text)
        tags = offsets_to_biluo_tags(doc, annot["Entities"]) # we add the bilou tags to words
        ents = biluo_tags_to_spans(doc, tags) # then find their spans
        doc.ents = ents
        db.add(doc)
    except IndexError:
        pass #if any statement is giving error pass will not consider that
    db.to_disk("validation.spacy")

In [3]:
#now we use transsfer learning
! python -m spacy init fill-config base_config.cfg config.cfg


[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [3]:
! python -m spacy debug data config.cfg


============================ Data file validation ============================
[+] Pipeline can be initialized with data
[+] Corpus is loadable

=============================== Training stats ===============================
Language: en
Training pipeline: tok2vec, ner
10000 training docs
10000 evaluation docs
[!] 9346 training examples also in evaluation data

============================== Vocab & Vectors ==============================
[i] 149874 total word(s) in the data (4138 unique)
[i] No word vectors present in the package

========================== Named Entity Recognition ==========================
[i] 1 label(s)
0 missing value(s) (tokens with '-' label)
[+] Good amount of examples for all labels
[+] Examples without occurrences available for all labels
[+] No entities consisting of or starting/ending with whitespace
[+] No entities crossing sentence boundaries

================================== Summary ==================================
[+] 6 checks passed
[!] 1 warning


In [6]:
! python -m spacy train "C:/Users/lexus/Desktop/DGI/week5/config.cfg" --output ./output

[2023-06-03 19:21:26,093] [INFO] Set up nlp object from config
[2023-06-03 19:21:26,109] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-06-03 19:21:26,109] [INFO] Created vocabulary
[2023-06-03 19:21:26,109] [INFO] Finished initializing nlp object
[2023-06-03 19:21:50,148] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


[+] Created output directory: output
[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     38.17    0.00    0.00    0.00    0.00
  0     200         46.35    901.26   94.15   97.92   90.66    0.94
  0     400         75.70    154.39   96.63   97.50   95.78    0.97
  0     600         83.76    116.41   96.40   97.29   95.53    0.96
  0     800        148.40    139.66   98.12   99.08   97.17    0.98
  0    1000        137.27    123.49   97.63   97.47   97.79    0.98
  0    1200        193.67    134.02   97.70   97.94   97.46    0.98
  0    1400        208.66    148.33   98.20   98.08 

In [9]:
#loading the model
recepie_model = spacy.load("C:/Users/lexus/Desktop/DGI/week5/output/model-best")

In [13]:
#predicting the ingredients
recepie1 = "in a medium frying pan , cook the bacon over medium heat until cooked but not crispy , 10 to 12 minutes , stirring frequently', 'remove the bacon to a paper towel-lined plate or pan , to remove excess fat', 'in a large bowl , whisk together the flour , sugar , baking powder , baking soda and salt', 'using a pastry cutter or fork , cut in the diced butter , until it resembles small peas', 'stir in the bacon , then one-fourth cup plus 2 tablespoons of maple syrup and the buttermilk until the dough just comes together', 'be careful not to overwork the dough', 'on a lightly floured surface , gently press or roll the dough to 1-inch thickness', 'cut the biscuits using a 2-inch round cutter', 'you should have 24 biscuits', 'place 12 biscuits on each of two parchment-lined baking sheets , spaced 2 inches apart', 'freeze the trays just until the biscuits are chilled , about 10 minutes', 'heat the oven to 350 degrees', 'while the biscuits are chilling , prepare the egg wash: in a small bowl , whisk together the egg yolk , egg and cream', 'brush the chilled biscuits with egg wash and top each with a pinch of fleur de sel', 'bake the biscuits until they just begin to brown , about 25 minutes', 'remove the tray from the oven', 'quickly drizzle 1 teaspoon of the remaining maple syrup over each biscuit , then place the tray back in the oven for 3 minutes more', 'bake from frozen: heat oven to 400 degrees', 'place desired number of frozen biscuits on a lightly oiled baking sheet', 'bake for 20-26 minutes to desired browning is achieved"
doc1 = recepie_model(recepie1)
ingredients1 = [ent.text for ent in doc1.ents]
print(recepie1, "\n\n", "Ingredients:", ingredients1)


in a medium frying pan , cook the bacon over medium heat until cooked but not crispy , 10 to 12 minutes , stirring frequently', 'remove the bacon to a paper towel-lined plate or pan , to remove excess fat', 'in a large bowl , whisk together the flour , sugar , baking powder , baking soda and salt', 'using a pastry cutter or fork , cut in the diced butter , until it resembles small peas', 'stir in the bacon , then one-fourth cup plus 2 tablespoons of maple syrup and the buttermilk until the dough just comes together', 'be careful not to overwork the dough', 'on a lightly floured surface , gently press or roll the dough to 1-inch thickness', 'cut the biscuits using a 2-inch round cutter', 'you should have 24 biscuits', 'place 12 biscuits on each of two parchment-lined baking sheets , spaced 2 inches apart', 'freeze the trays just until the biscuits are chilled , about 10 minutes', 'heat the oven to 350 degrees', 'while the biscuits are chilling , prepare the egg wash: in a small bowl , w

In [15]:
#loading the pickle model
import pickle
#saving the model as a pickle file
pickle.dump(recepie_model, open('model.pkl','wb'))

In [16]:
# Loading model to compare the results
model = pickle.load(open('model.pkl','rb'))
recepie2 = "saute black walnuts in the 3 tbs', 'of butter & let cool', 'combine sugar , flour & salt , slowly stir in hot milk', 'cook 10 minutes over low heat , stirring constantly', 'stir small amount of cooked mixture into beaten eggs , return to milk mixture & cook 1 mniute', 'chill in refrigerator , then pour in ice cream freezer & add cream & vanilla', 'churn in freezer for about 15 minutes , then add black walnuts & finish freezing"
doc2 = model(recepie2)
ingredients2 = [ent.text for ent in doc2.ents]
print(recepie2, "\n\n", "Ingredients:", ingredients2)


saute black walnuts in the 3 tbs', 'of butter & let cool', 'combine sugar , flour & salt , slowly stir in hot milk', 'cook 10 minutes over low heat , stirring constantly', 'stir small amount of cooked mixture into beaten eggs , return to milk mixture & cook 1 mniute', 'chill in refrigerator , then pour in ice cream freezer & add cream & vanilla', 'churn in freezer for about 15 minutes , then add black walnuts & finish freezing 

 Ingredients: ['black walnuts', 'butter', 'sugar', 'flour', 'salt', 'milk', 'eggs', 'milk', 'ice cream', 'cream', 'vanilla', 'black walnuts']
[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  -----

[2023-06-03 19:21:58,254] [INFO] Set up nlp object from config
[2023-06-03 19:21:58,275] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-06-03 19:21:58,281] [INFO] Created vocabulary
[2023-06-03 19:21:58,283] [INFO] Finished initializing nlp object
[2023-06-03 19:22:08,056] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
